In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import numpy as np
import glob
import os
import json
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
%aimport utils

In [3]:
pretrained_model = InceptionV3(weights='imagenet')

95739904/96112376 [============================>.] - ETA: 0s

In [4]:
pretrained_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [10]:
model1 = Model(pretrained_model.inputs, [pretrained_model.get_layer('avg_pool').output])
model2 = Model(pretrained_model.inputs, [pretrained_model.get_layer('predictions').output])

In [11]:
image_files = glob.glob("/mnt/workspace/Ugallery/images/*.jpg")

In [12]:
id2url_map = utils.get_id2url_map()

In [13]:
artwork_ids = [int(f[f.rfind('/')+1:-4]) for f in image_files]

In [15]:
N = len(artwork_ids)
N

13297

In [17]:
featmat_avg_pool = np.empty(shape=(N,2048))
featmat_predictions = np.empty(shape=(N,1000))

In [18]:
i_start = 0
i_end = N
utils.process_image_batch(
    [model1, model2], [featmat_avg_pool, featmat_predictions], image_files, artwork_ids, id2url_map,
    i_start, i_end, preprocess_input, image_target_size=(299, 299))

In [19]:
featmat_avg_pool.mean(), featmat_predictions.mean()

(0.35205834624024485, 0.001000000000078792)

In [21]:
dir_path = "/mnt/workspace/Ugallery/InceptionV3/"
os.makedirs(dir_path, exist_ok=True)
featmat_avg_pool.dump(dir_path + "avg_pool.npy")
featmat_predictions.dump(dir_path + "predictions.npy")
with open(dir_path + 'ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%d\n' % _id)